In [2]:
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import Sequential
from datetime import datetime
from PIL import Image
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import mlflow
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.models import Model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory
dataset_dir = "dataset_2_final"



In [4]:
# Set image and batch size
image_size = (300, 300)
batch_size = 4

# Create the ImageDataGenerator with data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting the dataset into training and validation sets
)

# Load the training dataset using image_dataset_from_directory
train_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=True,
    subset='training',    # Selecting the training set
    seed=1337
)

# Load the validation dataset using image_dataset_from_directory
val_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=False,        # No need to shuffle the validation set
    subset='validation',  # Selecting the validation set
    seed=1337
)


Found 423 images belonging to 7 classes.
Found 102 images belonging to 7 classes.


In [5]:
# Check the number of classes in your dataset
num_classes = len(train_ds.class_indices)
print(num_classes)

7


EfficientNetB0: Image input size is (224, 224, 3)
EfficientNetB1: Image input size is (240, 240, 3)
EfficientNetB2: Image input size is (260, 260, 3)
EfficientNetB3: Image input size is (300, 300, 3)
EfficientNetB4: Image input size is (380, 380, 3)
EfficientNetB5: Image input size is (456, 456, 3)
EfficientNetB6: Image input size is (528, 528, 3)
EfficientNetB7: Image input size is (600, 600, 3)

In [6]:
# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [9]:
# Train the model
epochs = 60
steps_per_epoch = len(train_ds)
validation_steps = len(val_ds)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/60
106/106 [==============================] - 71s 664ms/step - loss: 0.7817 - accuracy: 0.7258 - val_loss: 2.3674 - val_accuracy: 0.4902
Epoch 2/60
106/106 [==============================] - 69s 653ms/step - loss: 0.7571 - accuracy: 0.7565 - val_loss: 1.9487 - val_accuracy: 0.3824
Epoch 3/60
106/106 [==============================] - 69s 651ms/step - loss: 0.7303 - accuracy: 0.7636 - val_loss: 1.6610 - val_accuracy: 0.5392
Epoch 4/60
106/106 [==============================] - 70s 657ms/step - loss: 0.6214 - accuracy: 0.7849 - val_loss: 3.1823 - val_accuracy: 0.4510
Epoch 5/60
106/106 [==============================] - 69s 648ms/step - loss: 0.6579 - accuracy: 0.7565 - val_loss: 1.3529 - val_accuracy: 0.5588
Epoch 6/60
106/106 [==============================] - 69s 652ms/step - loss: 0.7108 - accuracy: 0.7589 - val_loss: 2.3286 - val_accuracy: 0.4608
Epoch 7/60
106/106 [==============================] - 69s 652ms/step - loss: 0.6938 - accuracy: 0.7447 - val_loss: 1.6883 - val_ac